# Data collection

Using usgs API that is built int to the dataretrieval module to collect real time water quality data 
https://waterservices.usgs.gov/

In [16]:
# Import required packages
import dataretrieval.nwis as nwis
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

#LAKE MOULTRIE TAILRACE CANAL AT MONCKS CORNER, SC
site_id = '02172002'


params = { #USGS parameter codes for water quality parameters
    '00010' : 'Temperature',
    '00095' : 'Specific Conductance (uS/cm)',
    '00300': 'Dissolved Oxygen (mg/L)',
    '00400': 'pH'
}
end_date = datetime.now()
start_date = end_date - timedelta(hours=24)

df = nwis.get_iv(
    sites=site_id,
    parameterCd=list(params.keys()),
    start=start_date.strftime('%Y-%m-%dT%H:%M'),
    end=end_date.strftime('%Y-%m-%dT%H:%M')
)
data = df[0]
metadata = df[1]


data.head()


,site_no,00010,00010_cd,00300,00300_cd
datetime,,,,,
2026-02-14 22:30:00+00:00,02172002,8.7,P,12.0,P
2026-02-14 22:45:00+00:00,02172002,8.7,P,12.0,P
2026-02-14 23:00:00+00:00,02172002,8.6,P,12.0,P
2026-02-14 23:15:00+00:00,02172002,8.7,P,12.0,P
2026-02-14 23:30:00+00:00,02172002,8.8,P,12.0,P


**Identifying site for project Demo**

As the current worst rated state for water quality and eco-friendliness, ive limited the site search to within this state

In terms of identifying the site, I am searching for sites that monitor temperature, conductance, streamflow, DO and PH.

**Parameter Reasoning**

Temperature (code 00010): Rapid changes can stress ecosystems and favor algal blooms

Streamflow (code 00060): Sudden spikes or drops in flow can indicate storm runoff or unusual releases; affects dilution

Electrical conductivity (code 00095): Surrogate for total dissolved solids; spikes may indicate runoff or effluent

pH (code 00400): Sensitive to chemical contamination or algal activity

Dissolved Oxygen (00300): Low DO can indicate nutrient overloading or organic pollution


In [ ]:
import requests
url = "https://waterservices.usgs.gov/nwis/iv/"
params = {
    "format": "json",
    "parameterCd": "00010,00060,00095,00300,00400",  # temp, discharge, conductance, DO, pH
    "siteType": "ST",      # stream sites
    "siteStatus": "active",
    "stateCd": "LA"        # Louisiana as example
}

response = requests.get(url, params=params)
data = response.json()

sites_dict = {}

for ts in data.get("value", {}).get("timeSeries", []):
    site_id = ts["sourceInfo"]["siteCode"][0]["value"]
    param = ts["variable"]["variableCode"][0]["value"]
    
    if site_id not in sites_dict:
        sites_dict[site_id] = set()
    sites_dict[site_id].add(param)

# Filter sites that have all five parameters
desired_params = {"00010", "00060", "00095", "00300", "00400"}
sites_with_all = [site for site, params in sites_dict.items() if desired_params.issubset(params)]

print("Sites with all five parameters:", sites_with_all)



Sites with all five parameters: ['07374000', '07381600']


**Identified Sites**

sites '07374000' and '07381600'
